In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [3]:
!wget -q https://dlcdn.apache.org/spark/spark-3.1.3/spark-3.1.3-bin-hadoop2.7.tgz

In [4]:
!tar xf spark-3.1.3-bin-hadoop2.7.tgz

In [5]:
!pip install -q findspark

In [6]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.3-bin-hadoop2.7"

In [7]:
import findspark
findspark.init()

In [8]:
findspark.find()

'/content/spark-3.1.3-bin-hadoop2.7'

In [9]:
from pyspark.sql import SparkSession

spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '4050')\
        .getOrCreate()

In [10]:
spark

In [11]:
#ilovepandas
import pandas as pd
#https://catalog.data.gov/dataset/monthly-counts-of-deaths-by-select-causes-2014-2019-da9df
mcod_url = 'https://data.cdc.gov/api/views/bxq8-mugm/rows.csv?accessType=DOWNLOAD'

df = pd.read_csv(mcod_url, sep=",")
df.columns = df.columns.str.replace('[ ()-,]', '')
df.to_csv("mocd.csv", index=False)
df.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: FutureWarning: The default value of regex will change from True to False in a future version.
  


,JurisdictionofOccurrence,Year,Month,AllCause,NaturalCause,Septicemia,MalignantNeoplasms,DiabetesMellitus,AlzheimerDisease,InfluenzaandPneumonia,...,OtherDiseasesofRespiratorySystem,NephritisNephroticSyndromeandNephrosis,SymptomsSignsandAbnormalClinicalandLaboratoryFindingsNotElsewhereClassified,DiseasesofHeart,CerebrovascularDiseases,AccidentsUnintentionalInjuries,MotorVehicleAccidents,IntentionalSelf-HarmSuicide,AssaultHomicide,DrugOverdose
0,United States,2014,1,243298,226621,3944,51101,7344,8305,7929,...,3466,4600,2815,58229,12074,11461,2572,3320,1213,4026
1,United States,2015,1,265355,247269,4194,52346,8053,11638,10005,...,3797,4979,3005,63190,13576,12311,2754,3618,1437,4354
2,United States,2016,1,245823,227341,3846,51863,7392,10612,5295,...,3705,4645,2755,58049,12968,12559,2734,3720,1499,4631
3,United States,2017,1,262832,241918,4089,52120,7907,12018,6925,...,4083,4818,2769,61650,13595,14520,3034,3709,1726,6233
4,United States,2018,1,286744,265418,4502,52876,8674,13410,12164,...,4603,5346,3138,67024,14653,14748,3010,3966,1674,5659


In [12]:
mocd = spark.read.csv("/content/mocd.csv", header=True, inferSchema=True, sep=",")
mocd.toPandas()

,JurisdictionofOccurrence,Year,Month,AllCause,NaturalCause,Septicemia,MalignantNeoplasms,DiabetesMellitus,AlzheimerDisease,InfluenzaandPneumonia,...,OtherDiseasesofRespiratorySystem,NephritisNephroticSyndromeandNephrosis,SymptomsSignsandAbnormalClinicalandLaboratoryFindingsNotElsewhereClassified,DiseasesofHeart,CerebrovascularDiseases,AccidentsUnintentionalInjuries,MotorVehicleAccidents,IntentionalSelf-HarmSuicide,AssaultHomicide,DrugOverdose
0,United States,2014,1,243298,226621,3944,51101,7344,8305,7929,...,3466,4600,2815,58229,12074,11461,2572,3320,1213,4026
1,United States,2015,1,265355,247269,4194,52346,8053,11638,10005,...,3797,4979,3005,63190,13576,12311,2754,3618,1437,4354
2,United States,2016,1,245823,227341,3846,51863,7392,10612,5295,...,3705,4645,2755,58049,12968,12559,2734,3720,1499,4631
3,United States,2017,1,262832,241918,4089,52120,7907,12018,6925,...,4083,4818,2769,61650,13595,14520,3034,3709,1726,6233
4,United States,2018,1,286744,265418,4502,52876,8674,13410,12164,...,4603,5346,3138,67024,14653,14748,3010,3966,1674,5659
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,United States,2015,12,233735,215657,3517,50462,6860,10112,4449,...,3353,4365,3211,55220,12260,12389,3245,3469,1568,4246
68,United States,2016,12,249207,228888,3714,51428,7446,11277,4981,...,3620,4472,3465,58572,13320,14363,3330,3397,1609,5829
69,United States,2017,12,258657,237819,3859,52001,7983,11751,6058,...,3816,4730,3317,60902,13740,14728,3391,3629,1580,5799
70,United States,2018,12,249280,229184,3620,51622,7663,10950,4489,...,3899,4647,3104,58613,13111,13851,3181,3713,1556,5551


In [13]:
mocd.printSchema()

root
 |-- JurisdictionofOccurrence: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Month: integer (nullable = true)
 |-- AllCause: integer (nullable = true)
 |-- NaturalCause: integer (nullable = true)
 |-- Septicemia: integer (nullable = true)
 |-- MalignantNeoplasms: integer (nullable = true)
 |-- DiabetesMellitus: integer (nullable = true)
 |-- AlzheimerDisease: integer (nullable = true)
 |-- InfluenzaandPneumonia: integer (nullable = true)
 |-- ChronicLowerRespiratoryDiseases: integer (nullable = true)
 |-- OtherDiseasesofRespiratorySystem: integer (nullable = true)
 |-- NephritisNephroticSyndromeandNephrosis: integer (nullable = true)
 |-- SymptomsSignsandAbnormalClinicalandLaboratoryFindingsNotElsewhereClassified: integer (nullable = true)
 |-- DiseasesofHeart: integer (nullable = true)
 |-- CerebrovascularDiseases: integer (nullable = true)
 |-- AccidentsUnintentionalInjuries: integer (nullable = true)
 |-- MotorVehicleAccidents: integer (nullable = true)
 |-

In [14]:
mocd.count()

72

In [15]:
mocd.select("Month", "AllCause", "NaturalCause", "Septicemia", "DiseasesofHeart").show(5)

+-----+--------+------------+----------+---------------+
|Month|AllCause|NaturalCause|Septicemia|DiseasesofHeart|
+-----+--------+------------+----------+---------------+
|    1|  243298|      226621|      3944|          58229|
|    1|  265355|      247269|      4194|          63190|
|    1|  245823|      227341|      3846|          58049|
|    1|  262832|      241918|      4089|          61650|
|    1|  286744|      265418|      4502|          67024|
+-----+--------+------------+----------+---------------+
only showing top 5 rows



In [16]:
mocd.describe().show()

+-------+------------------------+-----------------+-----------------+------------------+------------------+-----------------+------------------+-----------------+------------------+---------------------+-------------------------------+--------------------------------+--------------------------------------+---------------------------------------------------------------------------+-----------------+-----------------------+------------------------------+---------------------+---------------------------+------------------+-----------------+
|summary|JurisdictionofOccurrence|             Year|            Month|          AllCause|      NaturalCause|       Septicemia|MalignantNeoplasms| DiabetesMellitus|  AlzheimerDisease|InfluenzaandPneumonia|ChronicLowerRespiratoryDiseases|OtherDiseasesofRespiratorySystem|NephritisNephroticSyndromeandNephrosis|SymptomsSignsandAbnormalClinicalandLaboratoryFindingsNotElsewhereClassified|  DiseasesofHeart|CerebrovascularDiseases|AccidentsUnintentionalInjurie

In [17]:
mocd.groupBy('Year').sum().select("Year", "sum(AllCause)", "sum(NaturalCause)", "sum(Septicemia)" , "sum(DiseasesofHeart)").show()

+----+-------------+-----------------+---------------+--------------------+
|Year|sum(AllCause)|sum(NaturalCause)|sum(Septicemia)|sum(DiseasesofHeart)|
+----+-------------+-----------------+---------------+--------------------+
|2018|      2839205|          2594018|          40718|              655381|
|2015|      2712630|          2495936|          40773|              633842|
|2014|      2626418|          2424126|          38940|              614348|
|2019|      2854838|          2603461|          38431|              659041|
|2016|      2744248|          2509054|          40613|              635260|
|2017|      2813503|          2566005|          40922|              647457|
+----+-------------+-----------------+---------------+--------------------+



In [18]:
from pyspark.sql import functions as F
mocd.groupBy("Year").agg(F.avg("NaturalCause"), (F.sum('Septicemia') / F.sum('AllCause')).alias("perc of Septicemia")).show()

+----+------------------+--------------------+
|Year| avg(NaturalCause)|  perc of Septicemia|
+----+------------------+--------------------+
|2018|216168.16666666666|0.014341338508490934|
|2015|207994.66666666666|0.015030800367171342|
|2014|          202010.5|0.014826276700814569|
|2019|216955.08333333334|0.013461709561102942|
|2016|209087.83333333334|0.014799318428946655|
|2017|         213833.75|0.014544857425067611|
+----+------------------+--------------------+

